# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [1]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [2]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLutSimple'
save_path             = os.path.join(data_path, net_name)
rtl_sim_path          = '../../verilog/mnist'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 0
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズは64です。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [3]:
# dataset
dataset_train = torchvision.datasets.MNIST(root=data_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=data_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=64, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=64, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。

In [4]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([1024]),
            bb.DifferentiableLut([420]),
            bb.DifferentiableLut([70]),
            bb.Reduce([10]),
            bb.BinaryToReal(frame_modulation_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [5]:
#bb.load_networks(save_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(save_path, net, keep_olds=3)

## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [6]:
# export verilog
bb.load_networks(save_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net)

# Simulation用ファイルに上書きコピー
shutil.copyfile(output_velilog_file, sim_velilog_file)

load : ./data/MnistDifferentiableLutSimple\20201230_161118


'../../verilog/mnist\\MnistLutSimple.v'

In [7]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [8]:
# 最新の保存データ読み込み
bb.load_networks(save_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

load : ./data/MnistDifferentiableLutSimple\20201230_161118


### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x6 の行列が取得できます。

In [9]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(1024, 6)


array([[285, 227, 432, 459, 774, 502],
       [246,   4, 148, 633, 287, 638],
       [590, 679, 175, 507, 123, 196],
       ...,
       [652, 689, 385, 238, 444, 693],
       [214, 732, 514,  31,  30, 410],
       [548, 372, 435, 273,   8, 598]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [10]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(1024, 64)


array([[ True,  True,  True, ..., False,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False,  True, False, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [False, False, False, ...,  True,  True,  True],
       [ True,  True,  True, ..., False,  True, False]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [11]:
W = layer0.W().numpy()
print(W.shape)
W

(1024, 64)


array([[0.5319281 , 0.46951053, 0.5733587 , ..., 0.4717102 , 0.484694  ,
        0.5146377 ],
       [0.5179902 , 0.48048759, 0.5138343 , ..., 0.61111933, 0.59058636,
        0.60133845],
       [0.43197432, 0.45865867, 0.46974126, ..., 0.64330673, 0.57856894,
        0.60045767],
       ...,
       [0.52931947, 0.45074382, 0.5597966 , ..., 0.45929053, 0.4534356 ,
        0.4742707 ],
       [0.50331956, 0.4148569 , 0.47503656, ..., 0.5143429 , 0.55180293,
        0.538638  ],
       [0.4988206 , 0.499053  , 0.5786278 , ..., 0.37334788, 0.5298596 ,
        0.50942224]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [12]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(1024, 64)


array([[ 9.9345925e-04, -1.9961728e-03,  3.7461703e-04, ...,
        -3.9499150e-06,  2.8121725e-05, -1.0804986e-06],
       [-3.1896261e-04, -7.4131065e-05, -1.0632089e-04, ...,
        -1.5995120e-06,  7.7564864e-06, -5.3317171e-07],
       [-5.1385956e-05, -1.7128652e-05,  7.6244614e-05, ...,
         2.2666472e-07,  1.3529657e-06,  4.5098972e-07],
       ...,
       [ 5.6856754e-04,  2.0029891e-04,  7.1432802e-04, ...,
        -7.3252386e-06, -7.3903725e-06, -1.2641797e-06],
       [-1.5008255e-04,  1.2325025e-04, -5.0027509e-05, ...,
         2.3957109e-06, -3.1364116e-06,  7.9856989e-07],
       [-1.5965171e-04, -3.6072830e-04,  6.8319830e-05, ...,
        -2.2916847e-06, -1.7123117e-05, -5.1912986e-05]], dtype=float32)